In [4]:
import pandas as pd
import numpy as np

In [5]:
train = pd.read_csv('../data/train.csv')
train_labels = pd.read_csv('../data/train_labels.csv')
test = pd.read_csv('../data/test.csv')
sample_submission = pd.read_csv('../data/sample_submission.csv')

In [6]:
train.shape

(11341042, 11)

In [7]:
train.head()

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
0,27253bdc,45bb1e1b6b50c07b,2019-09-06T17:53:46.937Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE
1,27253bdc,17eeb7f223665f53,2019-09-06T17:54:17.519Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK
2,77261ab5,0848ef14a8dc6892,2019-09-06T17:54:56.302Z,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",0001e90f,1,2000,0,Sandcastle Builder (Activity),Activity,MAGMAPEAK
3,b2dba42b,0848ef14a8dc6892,2019-09-06T17:54:56.387Z,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,2,3010,53,Sandcastle Builder (Activity),Activity,MAGMAPEAK
4,1bb5fbdb,0848ef14a8dc6892,2019-09-06T17:55:03.253Z,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,3,3110,6972,Sandcastle Builder (Activity),Activity,MAGMAPEAK


In [ ]:
##Data Cleaning

In [36]:
assessed_ids = train[train['type'] == 'Assessment']['installation_id'].unique()
train = train[train['installation_id'].isin(assessed_ids)]
train.shape

(8294138, 11)

In [37]:
labeled_ids = train_labels['installation_id'].unique()
train = train[train['installation_id'].isin(labeled_ids)]
train.shape

(7734558, 11)

In [38]:
train['timestamp'] = pd.to_datetime(train['timestamp'])
train.sort_values(['installation_id', 'timestamp'], inplace=True)

In [39]:
train.head()

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
1538,27253bdc,34ba1a28d02ba8ba,2019-08-06 04:57:18.904000+00:00,"{""event_code"": 2000, ""event_count"": 1}",0006a69f,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE
1539,27253bdc,4b57c9a59474a1b9,2019-08-06 04:57:45.301000+00:00,"{""event_code"": 2000, ""event_count"": 1}",0006a69f,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK
1540,77261ab5,2b9d5af79bcdb79f,2019-08-06 04:58:14.538000+00:00,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",0006a69f,1,2000,0,Sandcastle Builder (Activity),Activity,MAGMAPEAK
1541,b2dba42b,2b9d5af79bcdb79f,2019-08-06 04:58:14.615000+00:00,"{""description"":""Let's build a sandcastle! Firs...",0006a69f,2,3010,29,Sandcastle Builder (Activity),Activity,MAGMAPEAK
1542,1325467d,2b9d5af79bcdb79f,2019-08-06 04:58:16.680000+00:00,"{""coordinates"":{""x"":273,""y"":650,""stage_width"":...",0006a69f,3,4070,2137,Sandcastle Builder (Activity),Activity,MAGMAPEAK


In [40]:
#total number of misses

total_number_of_misses = (train_labels['num_correct'].values == 0).sum()
total_number_of_misses

4229

In [41]:
def total_number_of_misses (df):
    total_number_of_misses = (train_labels['num_correct'].values == 0).sum()
    return total_number_of_misses

In [42]:
#percentage of misses - pretty convoluted

percentage_of_misses = ((train_labels['num_correct'].values == 0).sum())/(((train_labels['num_correct'].values == 1).sum())+((train_labels['num_correct'].values == 0).sum()))
percentage_of_misses

0.2390616167326173

In [43]:
def percentage_of_misses (df):
    percentage_misses = ((train_labels['num_correct'].values == 0).sum())/(((train_labels['num_correct'].values == 1).sum())+((train_labels['num_correct'].values == 0).sum()))
    return percentage_misses

In [44]:
from datetime import datetime, timedelta

In [45]:
start_time = datetime(2019, 9, 5, 7, 45).time()
start_time

datetime.time(7, 45)

In [46]:
end_time = datetime(2019, 9, 6, 15, 30).time()
end_time

datetime.time(15, 30)

In [47]:
def num_events_during_school_hours(df):
    start_time = datetime(2019, 9, 5, 7, 45).time()
    end_time = datetime(2019, 9, 6, 15, 30).time()
    df.index = df['timestamp']
    df = df.between_time(start_time, end_time).reset_index(drop=True)
    return len(df)

In [48]:
train_copy = train.copy()
train_copy.index = train_copy['timestamp']
train_copy.head()
train_copy.between_time(start_time, end_time).reset_index(drop=True)

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
0,27253bdc,a9fbd6119581ac51,2019-09-14 15:04:34.845000+00:00,"{""event_code"": 2000, ""event_count"": 1}",0006c192,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE
1,27253bdc,bbc818719db7d1f8,2019-09-14 15:04:55.292000+00:00,"{""event_code"": 2000, ""event_count"": 1}",0006c192,1,2000,0,Tree Top City - Level 1,Clip,TREETOPCITY
2,c7f7f0e1,e6298f19b82e25a4,2019-09-14 15:05:30.976000+00:00,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",0006c192,1,2000,0,Bug Measurer (Activity),Activity,TREETOPCITY
3,0a08139c,e6298f19b82e25a4,2019-09-14 15:05:31.196000+00:00,"{""description"":""walk_in Check it out! This is ...",0006c192,2,3010,246,Bug Measurer (Activity),Activity,TREETOPCITY
4,e79f3763,e6298f19b82e25a4,2019-09-14 15:05:32.986000+00:00,"{""bug"":""caterpillar"",""source"":""resources"",""coo...",0006c192,3,4030,2101,Bug Measurer (Activity),Activity,TREETOPCITY
...,...,...,...,...,...,...,...,...,...,...,...
1697633,15a43e5b,55ae180cd1d87496,2019-08-07 13:20:34.604000+00:00,"{""coordinates"":{""x"":400,""y"":305,""stage_width"":...",ffd2871d,331,4070,601961,Bottle Filler (Activity),Activity,MAGMAPEAK
1697634,15a43e5b,55ae180cd1d87496,2019-08-07 13:20:34.686000+00:00,"{""coordinates"":{""x"":483,""y"":302,""stage_width"":...",ffd2871d,332,4070,601961,Bottle Filler (Activity),Activity,MAGMAPEAK
1697635,15a43e5b,55ae180cd1d87496,2019-08-07 13:20:34.785000+00:00,"{""coordinates"":{""x"":350,""y"":303,""stage_width"":...",ffd2871d,333,4070,601961,Bottle Filler (Activity),Activity,MAGMAPEAK
1697636,15a43e5b,55ae180cd1d87496,2019-08-07 13:20:34.881000+00:00,"{""coordinates"":{""x"":482,""y"":302,""stage_width"":...",ffd2871d,334,4070,601961,Bottle Filler (Activity),Activity,MAGMAPEAK


In [49]:
train_copy.shape

(7734558, 11)

In [50]:
train_raw = train.copy()  # just in case we need the original data

train = pd.merge(train, train_labels, on=['installation_id', 'game_session'], how='left')

In [51]:
train_cuts = train[(train['event_code'] == 2000) & 
                   (train['type'] == 'Assessment') & 
                   (train['accuracy_group'].notnull())]

In [52]:
combos = train_cuts['installation_id'] + train_cuts['game_session']
combos.nunique(), combos.count()  # make sure there are no duplicates

(17690, 17690)

In [53]:
%%time

count = 0
X, y = [], []
for i, row in train_cuts.iterrows():
    count += 1
    installation_id, game_session = row['installation_id'], row['game_session']
    df = train[train['installation_id'] == installation_id]
    # get the timestamp of the cut row
    cut_time = df.loc[i,'timestamp']
    # cut the df
    df = df[df['timestamp'] <= cut_time]
    # cut off last row (assumes df is sorted by time)
    df = df.iloc[:-1,:]
    if df.empty:
        continue
    df['accuracy_group'] = row['accuracy_group']
    
    
    feature = {'installation_id': installation_id,
               'game_session': game_session,
               'num_events_during_school_hours': num_events_during_school_hours(df),
               'total_number_of_misses': total_number_of_misses(df),
               'percentage_of_misses': percentage_of_misses(df)}
    
    
    X.append(feature)
    y.append(row['accuracy_group'])
    
    if count % 1000 == 0:
        print('progress = {}%'.format(count/17690*100))
    
#     if count > 250:  # note we're just making features for the first n cuts
#         break

X = pd.DataFrame(X)
X.shape, len(y)

progress = 5.652911249293386%
progress = 11.305822498586773%
progress = 16.95873374788016%
progress = 22.611644997173546%
progress = 28.26455624646693%
progress = 33.91746749576032%
progress = 39.5703787450537%
progress = 45.22328999434709%
progress = 50.87620124364047%
progress = 56.52911249293386%
progress = 62.182023742227244%
progress = 67.83493499152064%
progress = 73.48784624081401%
progress = 79.1407574901074%
progress = 84.7936687394008%
progress = 90.44657998869418%
progress = 96.09949123798756%
Wall time: 2h 53min 30s


((17577, 5), 17577)

In [54]:
X.to_csv('../features/conlon_features.csv', index=False)

In [24]:
X.shape

(17577, 5)